In [5]:
import osmnx as ox
import geopandas as gpd
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import requests 

In [8]:
# Function to find closest station to an address
def closest_station(address):
    geolocator = Nominatim(user_agent="geoapiJCO")
    location = geolocator.geocode(address)
    
    # Define a bounding box around Paris
    north, south, east, west = 48.9022, 48.8156, 2.4699, 2.2241

    # Use Overpass API to get metro stations in Paris
    query = f"""
    [out:json];
    (
    node["railway"="subway_entrance"]({south},{west},{north},{east});
    );
    out body;
    """
    response = requests.get("http://overpass-api.de/api/interpreter", params={'data': query})
    data = response.json()

    # Convert the data to a GeoDataFrame
    nodes = [{'id': element['id'], 'lat': element['lat'], 'lon': element['lon'], 'tags': element['tags']} for element in data['elements'] if element['type'] == 'node']
    df = gpd.GeoDataFrame(nodes)
    df['geometry'] = gpd.points_from_xy(df.lon, df.lat)

    # Find the closest station
    closest_station = None
    min_dist = float('inf')
    for idx, row in df.iterrows():
        station_location = (row['lat'], row['lon'])
        dist = geodesic((location.latitude, location.longitude), station_location).meters
        if dist < min_dist:
            min_dist = dist
            closest_station = row['tags'].get('name', 'No name')

    return closest_station


In [9]:

# Test the function
address = "10 Rue du Colisée, 75008 Paris"
print(closest_station(address))


Champs-Élysées


C:\Users\juan_\AppData\Local\Temp\ipykernel_25880\332674046.py:23: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  df['geometry'] = gpd.points_from_xy(df.lon, df.lat)
